<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2024-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 2. Metodos para Ecuaciones Diferenciales Parciales Elipticas<br> </h2>
<h3 style="color:#03122E;text-align:center;">             Metodos de Elementos Finitos <br> </h3>
<h3 style="color:#03122E;text-align:center;">             Continuum Mechanics <br> </h3>
<hr style="border:3px solid #E3AE00 "> </hr>

In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.geom2d import SplineGeometry
from pandas import DataFrame

In [2]:
def stress(u):
    return lambda_par*(div(u))*Id(2) + 2*mu_par*strain(u)

def strain(u):
    return 0.5*(grad(u) + grad(u).trans)

In [3]:
geo = SplineGeometry()

pnts =[(0,0),(5,0),(5,1),(0,1)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["line",p3,p4],"top"],
          [["line",p4,p1],"left"]]
[geo.Append(c,bc=bc, leftdomain=1, rightdomain=0) for c,bc in curves]
geo.AddCircle(c=(2.5,0.5),r=0.25,maxh=0.05,bc="circle",leftdomain=0,rightdomain=1)
ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)
Draw(mesh)

bdry_values = {'left': CoefficientFunction((0,0)), 'right': CoefficientFunction((1,0))}
values_list = [bdry_values[bc]
               if bc in bdry_values.keys() else ((0,0))
               for bc in mesh.GetBoundaries()]
g = CoefficientFunction(values_list)

lambda_par = Parameter(1.0) # Ratio lambda/mu = 1
mu_par = Parameter(1.0)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [4]:
# H1-conforming finite element space
fes = VectorH1(mesh, order=1, dirichlet='left|right')

# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()

# the right hand side
force = CoefficientFunction((0,0))

lf = LinearForm(fes)
lf += force * v * dx

# the bilinear-form 
a = BilinearForm(fes, symmetric=True)
a += lambda_par*div(u)*div(v)*dx + 2*mu_par*InnerProduct(strain(u),strain(v))*dx

a.Assemble()
lf.Assemble()

# the solution field 
gfu = GridFunction(fes)
gfu.Set(g, definedon=mesh.Boundaries('left|right'))
r = lf.vec.CreateVector()
r.data = lf.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r


In [5]:
Draw(gfu, deformation=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

# Perdida de la Coercividad

In [6]:
geo = SplineGeometry()

pnts =[(0,0),(3,0),(3,1),(0,1)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
          [["line",p2,p3],"right"],
          [["line",p3,p4],"top"],
          [["line",p4,p1],"left"]]
[geo.Append(c,bc=bc, leftdomain=1, rightdomain=0) for c,bc in curves]
geo.AddCircle(c=(1.5,0.5),r=0.25,maxh=0.025,bc="circle2",leftdomain=0,rightdomain=1)
geo.AddCircle(c=(0.625,0.5),r=0.2,maxh=0.025,bc="circle1",leftdomain=0,rightdomain=1)
geo.AddCircle(c=(2.375,0.5),r=0.2,maxh=0.025,bc="circle3",leftdomain=0,rightdomain=1)

ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)
Draw(mesh)

bdry_values = {'left': CoefficientFunction((0,0)), 'right': CoefficientFunction((1,0))}
values_list = [bdry_values[bc]
               if bc in bdry_values.keys() else ((0,0))
               for bc in mesh.GetBoundaries()]
g = CoefficientFunction(values_list)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [7]:
lambda_par = Parameter(100.0) # Ratio lambda/mu = 100
mu_par = Parameter(1.0)
def stress(u):
    return lambda_par*(div(u))*Id(2) + 2*mu_par*strain(u)

def strain(u):
    return 0.5*(grad(u) + grad(u).trans)
# H1-conforming finite element space
fes = VectorH1(mesh, order=1, dirichlet='left|right')

# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()

# the right hand side
force = CoefficientFunction((0,0))

lf = LinearForm(fes)
lf += force * v * dx

# the bilinear-form 
a = BilinearForm(fes, symmetric=True)
a += lambda_par*div(u)*div(v)*dx + 2*mu_par*InnerProduct(strain(u),strain(v))*dx

a.Assemble()
lf.Assemble()

# the solution field 
gfu = GridFunction(fes)
gfu.Set(g, definedon=mesh.Boundaries('left|right'))
r = lf.vec.CreateVector()
r.data = lf.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r


In [8]:
Draw(gfu, deformation=True)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [9]:
sigma = grad(gfu)#.MakeVariable()

def sigma_min(sigma): # magnitude of first principal stress
    return((sigma[0,0]+sigma[1,1])/2 - sqrt(((sigma[0,0]-sigma[1,1])/2)**2 + sigma[1,0]**2))

def sigma_max(sigma): # magnitude of first principal stress
    return((sigma[0,0]+sigma[1,1])/2 + sqrt(((sigma[0,0]-sigma[1,1])/2)**2 + sigma[1,0]**2))

Draw(sigma_max(sigma), mesh)
Draw(sigma_min(sigma), mesh)
Tresca = sigma_max(sigma)-sigma_min(sigma)
Draw(Tresca, mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

Se grafican los esfuerzos tresca y maximos obtenidos. Se observa oscilaciones espureads de la solucion discreta, esto se conoce como **locking**. 

# Planar elasticity, locking

In [29]:
def linear_elasticity_problem():
    # Material data
    E = 3.0 # Young's modulus
    # nu = 0.25 # Poisson's ratio nu
    nu = 0.4999 # Poisson's ratio nu - near incompressible
    mu  = 1/2*(E / (1+nu)) # lame parameter mu
    lam = E * nu / ((1+nu)*(1-2*nu))# lame parameter lambda
    print(f"mu = {mu}, lambda = {lam}")
    # exact solution
    u1      = CoefficientFunction(-x**2*(x-1.0)**2*y*(y-1.0)*(2.0*y-1.0) )
    u2      = -u1
    u       = CoefficientFunction((u1, u2))
    gradu   = 0
    e11     = -2.0*x*y*(2.0*x**2 - 3.0*x + 1.0)*(2*y**2 - 3.0*y + 1.0)
    e12     = -(x*(x - 1.0)*(6.0*x**2*y**2 - 6.0*x**2*y + x**2 - 8.0*x*y**3 + 6.0*x*y**2 + 2.0*x*y - x + 4.0*y**3 - 6.0*y**2 + 2.0*y))/2.0
    e22     = x**2*(x - 1.0)**2*(6.0*y**2 - 6.0*y + 1.0)
    epsilon = CoefficientFunction((e11,e12,e12,e22),  dims=(2,2)) # strain tensor
    
    # forces
    f1      = (E*(x - 2.0*y + 18.0*x**2*y**2 - 24.0*x**2*y**3 + 12.0*x**3*y**2 + 2.0*nu*y + 6.0*x*y - 6.0*nu*x**2 + 12.0*nu*x**3 - 6.0*nu*x**4 - 6.0*nu*y**2 + 4.0*nu*y**3 - 30.0*x*y**2 + 24.0*x*y**3 - 6.0*x**4*y - 4.0*x**3 + 3.0*x**4 + 6.0*y**2 - 4.0*y**3 + 36.0*nu*x*y**2 + 24.0*nu*x**2*y - 24.0*nu*x*y**3 - 24.0*nu*x**3*y + 12.0*nu*x**4*y - 36.0*nu*x**2*y**2 + 24.0*nu*x**2*y**3 - 12.0*nu*x*y))/(2.0*nu**2 + nu - 1.0)
    f2      = -(E*(x - y - 12.0*x**2*y**3 + 12.0*x**3*y**2 + 2*nu*y - 6*nu*x**2 + 12*nu*x**3 - 6*nu*x**4 - 6*nu*y**2 + 4*nu*y**3 - 12*x*y**2 + 12*x*y**3 + 12*x**3*y - 12*x**4*y + 3*x**2 - 10*x**3 + 6*x**4 + 3*y**2 - 2*y**3 + 36*nu*x*y**2 + 24*nu*x**2*y - 24*nu*x*y**3 - 24*nu*x**3*y + 12*nu*x**4*y - 36*nu*x**2*y**2 + 24*nu*x**2*y**3 - 12*nu*x*y))/(2.0*nu**2 + nu - 1.0)
    
    f       = CoefficientFunction((f1,f2))
    uD      = u
    
    # wrap it up
    material = {"E":E, "nu":nu, "mu":mu, "lamb":lam}
    exact = {"u":u, "strain":epsilon}
    data = {"bodyforce":f,"uD":uD}
    
    return exact, data, material

In [30]:
def epsilon(u):
    return 0.5*(grad(u)+grad(u).trans)

def sigma(u,mu,lamb):
    return lamb * div(u)*Id(2) + mu*(grad(u) + grad(u).trans)

In [31]:
def SetUP_linear_elasticity(mesh, data, material):
    
    Vh = VectorH1(mesh, order = 1, dirichlet='left|right|top|bottom')

    # variables
    u = Vh.TrialFunction()
    v = Vh.TestFunction()

    # bilinear form
    a = BilinearForm(Vh)
    a += (InnerProduct(sigma(u,material['mu'], material['lamb']),epsilon(v)))*dx
    a.Assemble()

    L = LinearForm(Vh)
    L += data['bodyforce']*v*dx 
    L.Assemble()

    # grid functions
    gfu = GridFunction(Vh, name="uh")

    
    return gfu, a, L

def Convergence_test(last_mesh=6):
    exact, data, material = linear_elasticity_problem()
    output = []
    for i in range(1, last_mesh):
        h = 2**(-i)
        
        mesh = Mesh(unit_square.GenerateMesh(maxh=h))
        
        gfu, a, L = SetUP_linear_elasticity(mesh, data, material)
        
        # set Dirichlet boudanry condition
        gfu.Set(data['uD'], BND)
        r = gfu.vec.CreateVector()
        r.data = L.vec - a.mat * gfu.vec

        # Solve linear system
        gfu.vec.data += a.mat.Inverse(freedofs=gfu.space.FreeDofs()) * r
        
        # output
        err_u       = sqrt( Integrate( InnerProduct( gfu - exact['u'], gfu - exact['u']), mesh))
        output.append ((h, gfu.space.ndof, err_u))
    # end for
    # Draw(gfu, mesh, 'displacement')
    return output

In [32]:
def print_markdown_table(errors):
    i = 1
    h = []
    N = []
    error_u = []
    order_u = []

    while i < len(errors):
        h.append(errors[i][0])
        N.append(errors[i][1])
        error_u.append(errors[i][2])
        order_u.append(log(errors[i-1][2]/errors[i][2])/log(errors[i-1][0]/errors[i][0]))
        i += 1

    # pretty print
    df = DataFrame({"h": h,"N": N,'$$\|u-u_h\|_{L^2}$$': error_u,"$$\mbox{order } u$$": order_u})
    display(df)

In [34]:
output = Convergence_test( last_mesh=9)
print_markdown_table(output)

mu = 1.0000666711114075, lambda = 4999.333288886476


,h,N,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$
0,0.250000,52,0.002212,inf
1,0.125000,178,0.001508,0.552637
2,0.062500,682,0.001565,-0.053185
3,0.031250,2530,0.001571,-0.005263
4,0.015625,9798,0.001105,0.506730
5,0.007812,38426,0.000491,1.169706
6,0.003906,152724,0.000154,1.671378


In [19]:
from netgen.csg import *

wi = 0.2
le = 1.0
geo = CSGeometry()
left  = Plane (Pnt(0,0.5*wi,0.5*wi), Vec(-1,0,0) ).bc('fix')
right = Plane (Pnt(1*le,0.5*wi,0.5*wi), Vec( 1,0,0) ).bc('free')
front = Plane (Pnt(0.5*le,0,0.5*wi), Vec(0,-1,0) ).bc('free')
back  = Plane (Pnt(0.5*le,1*wi,0.5*wi), Vec(0, 1,0) ).bc('free')
bot   = Plane (Pnt(0.5*le,0.5*wi,0), Vec(0,0,-1) ).bc('free')
top   = Plane (Pnt(0.5*le,0.5*wi,1*wi), Vec(0,0, 1) ).bc('free')

beam = left * right * front * back * bot * top 
geo.Add (beam)

0

In [20]:
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
# Draw (mesh)

mu = 1
rho = 1
delta = wi/le
gamma = 0.4*delta**2
beta = 1.25
lamb = beta
g = gamma

bodyforce = CoefficientFunction((0,0,-rho*g))
uD = CoefficientFunction((0,0,0)) 

In [21]:
def epsilon(u):
    return 0.5*(grad(u) + grad(u).trans)

def sigma(u):
    return lamb * div(u)*Id(mesh.dim) + mu*(grad(u) + grad(u).trans)
Vh = VectorH1(mesh, order = 1, dirichlet='fix')

# variables
u = Vh.TrialFunction()
v = Vh.TestFunction()

# bilinear form
a = BilinearForm(Vh)
a += (InnerProduct(sigma(u),epsilon(v)))*dx
a.Assemble()

L = LinearForm(Vh)
L += bodyforce*v*dx 
L.Assemble()

# grid functions
gfu = GridFunction(Vh, name="uh")

# set Dirichlet boudanry condition
gfu.Set(uD, BND)
L.vec.data -= a.mat * gfu.vec

# Solve linear system
solvers.BVP(bf=a, lf=L, gf=gfu)

In [22]:
Draw(gfu, mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene